# Computing Optimal Strategies for Naughts & Crosses via Brute Force

The problem is to 'play' all possible games in advance and store the results, such that when a player is presented with any state of the game they can choose as their next move the one with highest probability of leading to a win. Playing all possible games in this way is a 'brute force' approach to learning strategies. This algorithm will require:

1. A way to represent the state of a game that can also serve as an index to query against;
2. A method for identifying a winning state;
3. A method for computing the set of all possible moves conditional on the current state of the game; and,
4. An algorithm for playing all possible games and computing the probability of winning conditional on the current state of the game.

## Representing Game State

There are 9 cells in which a move can be made (the board is a 3 x 3 grid), and each cell can be in 1 of 3 possible states:

1. Player One has made a move in the cell (`"1"`);
2. Player Two has made a move in the cell (`"2"`); or,
3. No player has made a move in the cell (`"0"`).

In everything that follows Player One will always start and we will view the same entire from within their frame-of-reference (as the game is symmetrical between players) - i.e., we will assume that we are always Player One as far as computing win probabilities are concerned.

A viable way of representing state is as a 9 digit integer comprised solely of 3 values - e.g., the 3 x 3 (matrix) representation of the board

```text
102
012
100
```

Could just as easily be represented in a 'vector-like' setup,

```text
102012100
```

This can also be encoded as a string (to enable easy element lookup via its list protocol), that can also be used as an index to search against - e.g., as a dictionary key.

In [1]:
game_states = {
    "102012100": {
        "possible_moves": {
            "112012100": {"p_win": 0.01},
            "102112100": {"p_win": 0.02},
            "102012110": {"p_win": 0.03},
            "102012101": {"p_win": 0.04},
        }
    }
}

next_move_options = game_states["102012100"]["possible_moves"]
next_move_options

{'112012100': {'p_win': 0.01},
 '102112100': {'p_win': 0.02},
 '102012110': {'p_win': 0.03},
 '102012101': {'p_win': 0.04}}

## Identifying Winners

There are only 8 possible states that map to a win - 3 from the columns, 3 rom the rows, and 2 from the diagonals. We can check the state of the game explicitly against these.

In [2]:
def winning_state(game_state: str, player: str = "1") -> bool:
    """Check the game state to see if it indicates a winning state."""
    # check rows
    if game_state[0] == game_state[1] == game_state[2] == player:
        return True
    if game_state[3] == game_state[4] == game_state[5] == player:
        return True
    if game_state[6] == game_state[7] == game_state[8] == player:
        return True

    # check columns
    if game_state[0] == game_state[3] == game_state[6] == player:
        return True
    if game_state[1] == game_state[4] == game_state[7] == player:
        return True
    if game_state[2] == game_state[5] == game_state[8] == player:
        return True

    # check diagonals
    if game_state[0] == game_state[4] == game_state[8] == player:
        return True
    if game_state[6] == game_state[4] == game_state[8] == player:
        return True

    # otherwise...
    return False


print(f"111212122: win for Player One --> {winning_state('111212122', player='1')}")
print(f"120120100: win for Player One --> {winning_state('111212100', player='1')}")
print(f"120210001: win for Player One --> {winning_state('120210001', player='1')}")
print(f"201210100: win for Player One --> {winning_state('120210001', player='1')}")
print(f"112012100: win for Player One --> {winning_state('112012100', player='1')}")
print(f"222121211: win for Player Two --> {winning_state('222121211', player='2')}")
print(f"112102002: win for Player Two --> {winning_state('112102002', player='2')}")

111212122: win for Player One --> True
120120100: win for Player One --> True
120210001: win for Player One --> True
201210100: win for Player One --> True
112012100: win for Player One --> False
222121211: win for Player Two --> True
112102002: win for Player Two --> True


## Identifying Possible Moves

In [3]:
def get_possible_moves(game_state: str, player: str = "1") -> list[str]:
    """Compute all possible next moves."""
    possible_moves: list[str] = []
    for i in range(9):
        if game_state[i] == "0":
            possible_moves.append(game_state[:i] + player + game_state[i+1:])
    return possible_moves


get_possible_moves("120120100", player="1")

['121120100', '120121100', '120120110', '120120101']

## Playing all Possible Games and Computing `P(win|current_state)`

Start by playing all possible games.

In [4]:
players = ["1", "2"]
possible_games: list[list[str]] = []
possible_games.append(["000000000"])

# phase 1 - generate all possible games
for game_round in range(9):
    player = players[game_round % 2]
    new_possible_games: list[list[str]] = []
    for game in possible_games:
        previous_game_state = game[-1]
        if (winning_state(previous_game_state, player="1")
            or winning_state(previous_game_state, player="2")):
            new_possible_games.append(game.copy())
            continue
        for move in get_possible_moves(previous_game_state, player):
            new_game = game.copy()
            new_game.append(move)
            new_possible_games.append(new_game)
        possible_games = new_possible_games

print(f"number of possible games = {len(possible_games):,}")

number of possible games = 258,912


Let's look at some possible games.

In [5]:
def print_final_game_state(game_path: list[str]) -> None:
    """Print the game state on a 3 x 3 board."""
    final_game_state = game_path[-1]
    print(
        "final game state:" + "\n "
        + final_game_state[0:3] + "\n "
        + final_game_state[3:6] + "\n "
        + final_game_state[6:9]
    )


def print_game_path(game_path: list[str]) -> None:
    """Print the path of a finished game."""
    all_states_in_path = [state for state in game_path]
    print("game path: " + " --> ".join(all_states_in_path))


def print_game_result(game_path: list[str]) -> None:
    """Print the game result."""
    final_fame_state = game_path[-1]
    if winning_state(final_fame_state, "1"):
        print("result: player 1 wins")
    elif winning_state(final_fame_state, "2"):
        print("result: player 2 wins")
    else:
        print("result: draw")


game_path = 130
print_game_result(possible_games[game_path])
print_game_path(possible_games[game_path])
print_final_game_state(possible_games[game_path])

result: player 2 wins
game path: 000000000 --> 100000000 --> 120000000 --> 121000000 --> 121020000 --> 121021000 --> 121021200 --> 121121200 --> 121121220
final game state:
 121
 121
 220


Lets analyse the games.

In [6]:
def game_outcome(game_path: list[str]) -> str:
    """Compute the game's outcome."""
    final_fame_state = game_path[-1]
    if winning_state(final_fame_state, "1"):
        return ("player 1 wins")
    elif winning_state(final_fame_state, "2"):
        return ("player 2 wins")
    else:
        return("draw")


num_player_1_wins = sum(
    game_outcome(game) == "player 1 wins" for game in possible_games
)
num_player_2_wins = sum(
    game_outcome(game) == "player 2 wins" for game in possible_games
)
num_draws = sum(
    game_outcome(game) == "draw" for game in possible_games
)

print(f"number of times player 1 wins = {num_player_1_wins:,}")
print(f"number of times player 2 wins = {num_player_2_wins:,}")
print(f"number of draws = {num_draws:,}")

number of times player 1 wins = 120,888
number of times player 2 wins = 68,904
number of draws = 69,120


Now compute conditional probabilities - `P(win|current_state)`.

In [7]:
from collections import Counter

p1_win_counter: Counter[str] = Counter()
p2_win_counter: Counter[str]= Counter()
draw_counter: Counter[str] = Counter()
interim_states: set[str] = set()

outcome_probabilities: dict[str, dict[str, float]] = {}
ai_player_database: dict[str, dict[str, dict[str, float]]] = {}

# count wins and draws for every interim state in every game
for game in possible_games:
    final_game_state = game[-1]
    for game_state in game:  ## game[:-2] or game[:-1] analysis?
        interim_states.add(game_state)
        if winning_state(final_game_state, player="1"):
            p1_win_counter.update([game_state])
        elif winning_state(final_game_state, player="2"):
            p2_win_counter.update([game_state])
        else:
            draw_counter.update([game_state])

# compute outcome probabilities from every possible interim state
for game_state in interim_states:
    num_p1_wins_from_state = p1_win_counter[game_state]
    num_p2_wins_from_state = p2_win_counter[game_state]
    num_draws_from_state = draw_counter[game_state]

    num_final_outcomes = (
        num_p1_wins_from_state + num_p2_wins_from_state + num_draws_from_state
    )

    outcome_probabilities[game_state] = {
        "p1_wins": round(num_p1_wins_from_state / num_final_outcomes, 3),
        "p2_wins": round(num_p2_wins_from_state / num_final_outcomes, 3),
        "draw": round(num_draws_from_state / num_final_outcomes, 3)
    }

# compute data in a form that can be used for an AI player
for game_state in interim_states:
    if winning_state(game_state, player="1") or winning_state(game_state, player="2"):
        continue
    player = "1" if game_state.count("1") <= game_state.count("2") else "2"
    possible_moves_data: dict[str, dict[str, float]] = {}
    for possible_move in get_possible_moves(game_state, player):
        possible_moves_data[possible_move] = outcome_probabilities[possible_move]
    ai_player_database[game_state] = possible_moves_data

Let's take a look at some of the results!

In [8]:
from json import dumps

current_state = "000000000"
for possible_move, data in ai_player_database["000000000"].items():
    print(f"{current_state} --> {possible_move}: {dumps(data)}")

000000000 --> 100000000: {"p1_wins": 0.468, "p2_wins": 0.246, "draw": 0.286}
000000000 --> 010000000: {"p1_wins": 0.457, "p2_wins": 0.291, "draw": 0.252}
000000000 --> 001000000: {"p1_wins": 0.438, "p2_wins": 0.291, "draw": 0.271}
000000000 --> 000100000: {"p1_wins": 0.453, "p2_wins": 0.294, "draw": 0.252}
000000000 --> 000010000: {"p1_wins": 0.547, "p2_wins": 0.194, "draw": 0.259}
000000000 --> 000001000: {"p1_wins": 0.434, "p2_wins": 0.294, "draw": 0.272}
000000000 --> 000000100: {"p1_wins": 0.477, "p2_wins": 0.261, "draw": 0.262}
000000000 --> 000000010: {"p1_wins": 0.399, "p2_wins": 0.313, "draw": 0.288}
000000000 --> 000000001: {"p1_wins": 0.547, "p2_wins": 0.194, "draw": 0.259}
